In [2]:
import csv 
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("SpotifySongs").getOrCreate()

df = spark.read.csv("../stream-processing-template/assets/data/spotify-2023.csv", header=True, inferSchema=True)


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/12 13:32:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/12/12 13:32:40 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
/usr/local/lib/python3.7/site-packages/pyspark/context.py:317: FutureWarning: Python 3.7 support is deprecated in Spark 3.4.
  warnings.warn("Python 3.7 support is deprecated in Spark 3.4.", FutureWarning)


In [14]:
df.describe()

DataFrame[summary: string, track_name: string, artist(s)_name: string, artist_count: string, released_year: string, released_month: string, released_day: string, in_spotify_playlists: string, in_spotify_charts: string, streams: string, in_apple_playlists: string, in_apple_charts: string, in_deezer_playlists: string, in_deezer_charts: string, in_shazam_charts: string, bpm: string, key: string, mode: string, danceability_%: string, valence_%: string, energy_%: string, acousticness_%: string, instrumentalness_%: string, liveness_%: string, speechiness_%: string]

In [16]:
df.dtypes

[('track_name', 'string'),
 ('artist(s)_name', 'string'),
 ('artist_count', 'int'),
 ('released_year', 'int'),
 ('released_month', 'int'),
 ('released_day', 'int'),
 ('in_spotify_playlists', 'int'),
 ('in_spotify_charts', 'int'),
 ('streams', 'string'),
 ('in_apple_playlists', 'int'),
 ('in_apple_charts', 'int'),
 ('in_deezer_playlists', 'string'),
 ('in_deezer_charts', 'int'),
 ('in_shazam_charts', 'string'),
 ('bpm', 'int'),
 ('key', 'string'),
 ('mode', 'string'),
 ('danceability_%', 'int'),
 ('valence_%', 'int'),
 ('energy_%', 'int'),
 ('acousticness_%', 'int'),
 ('instrumentalness_%', 'int'),
 ('liveness_%', 'int'),
 ('speechiness_%', 'int')]

In [9]:
df.describe()

DataFrame[summary: string, track_name: string, artist(s)_name: string, artist_count: string, released_year: string, released_month: string, released_day: string, in_spotify_playlists: string, in_spotify_charts: string, streams: string, in_apple_playlists: string, in_apple_charts: string, in_deezer_playlists: string, in_deezer_charts: string, in_shazam_charts: string, bpm: string, key: string, mode: string, danceability_%: string, valence_%: string, energy_%: string, acousticness_%: string, instrumentalness_%: string, liveness_%: string, speechiness_%: string]

In [4]:
df.select("track_name", "artist(s)_name", "released_year", "streams").show()

+--------------------+--------------------+-------------+----------+
|          track_name|      artist(s)_name|released_year|   streams|
+--------------------+--------------------+-------------+----------+
|Seven (feat. Latt...|    Latto, Jung Kook|         2023| 141381703|
|                LALA|         Myke Towers|         2023| 133716286|
|             vampire|      Olivia Rodrigo|         2023| 140003974|
|        Cruel Summer|        Taylor Swift|         2019| 800840817|
|      WHERE SHE GOES|           Bad Bunny|         2023| 303236322|
|            Sprinter|   Dave, Central Cee|         2023| 183706234|
|     Ella Baila Sola|Eslabon Armado, P...|         2023| 725980112|
|            Columbia|             Quevedo|         2023|  58149378|
|            fukumean|               Gunna|         2023|  95217315|
|     La Bebe - Remix|Peso Pluma, Yng L...|         2023| 553634067|
|           un x100to|Bad Bunny, Grupo ...|         2023| 505671438|
|           Super Shy|            

In [28]:
from pyspark.sql.types import IntegerType

In [29]:
df = df.withColumn("streams", df["streams"].cast(IntegerType()))

In [31]:
df.filter(df.streams > 140000000).orderBy(df.streams.asc()).show()

+--------------------+--------------------+------------+-------------+--------------+------------+--------------------+-----------------+---------+------------------+---------------+-------------------+----------------+----------------+---+----+-----+--------------+---------+--------+--------------+------------------+----------+-------------+
|          track_name|      artist(s)_name|artist_count|released_year|released_month|released_day|in_spotify_playlists|in_spotify_charts|  streams|in_apple_playlists|in_apple_charts|in_deezer_playlists|in_deezer_charts|in_shazam_charts|bpm| key| mode|danceability_%|valence_%|energy_%|acousticness_%|instrumentalness_%|liveness_%|speechiness_%|
+--------------------+--------------------+------------+-------------+--------------+------------+--------------------+-----------------+---------+------------------+---------------+-------------------+----------------+----------------+---+----+-----+--------------+---------+--------+--------------+----------

In [5]:
df.filter(df.track_name == 'vampire').show()

+----------+--------------+------------+-------------+--------------+------------+--------------------+-----------------+---------+------------------+---------------+-------------------+----------------+----------------+---+---+-----+--------------+---------+--------+--------------+------------------+----------+-------------+
|track_name|artist(s)_name|artist_count|released_year|released_month|released_day|in_spotify_playlists|in_spotify_charts|  streams|in_apple_playlists|in_apple_charts|in_deezer_playlists|in_deezer_charts|in_shazam_charts|bpm|key| mode|danceability_%|valence_%|energy_%|acousticness_%|instrumentalness_%|liveness_%|speechiness_%|
+----------+--------------+------------+-------------+--------------+------------+--------------------+-----------------+---------+------------------+---------------+-------------------+----------------+----------------+---+---+-----+--------------+---------+--------+--------------+------------------+----------+-------------+
|   vampire|Oliv

In [35]:
df.groupBy("released_year").count().orderBy(df.released_year.desc()).show()

+-------------+-----+
|released_year|count|
+-------------+-----+
|         2023|  175|
|         2022|  402|
|         2021|  119|
|         2020|   37|
|         2019|   36|
|         2018|   10|
|         2017|   23|
|         2016|   18|
|         2015|   11|
|         2014|   13|
|         2013|   13|
|         2012|   10|
|         2011|   10|
|         2010|    7|
|         2008|    2|
|         2007|    1|
|         2005|    1|
|         2004|    4|
|         2003|    2|
|         2002|    6|
+-------------+-----+
only showing top 20 rows



In [42]:
df = df.withColumn("total_charts_presence", df.in_spotify_charts + df.in_apple_charts + df.in_deezer_charts + df.in_shazam_charts)

In [43]:
df.select("track_name", 'total_charts_presence').orderBy(df.total_charts_presence.desc()).show()

+--------------------+---------------------+
|          track_name|total_charts_presence|
+--------------------+---------------------+
|             vampire|               1283.0|
|            Sprinter|               1267.0|
|            fukumean|               1257.0|
|Seven (feat. Latt...|               1246.0|
|              Tattoo|               1104.0|
|     I'm Good (Blue)|               1036.0|
|  Baby Don't Hurt Me|                998.0|
|    The Next Episode|                994.0|
|        Cruel Summer|                867.0|
|Miracle (with Ell...|                810.0|
|         Eyes Closed|                777.0|
|         Bloody Mary|                763.0|
|           Moonlight|                737.0|
|Everybody Wants T...|                723.0|
|     Ella Baila Sola|                703.0|
|Set Fire to the Rain|                679.0|
|              People|                657.0|
|All My Life (feat...|                648.0|
|Somewhere Only We...|                643.0|
|      WHE

In [4]:
df.select(['track_name', 'in_spotify_ranking']).show()

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `in_spotify_ranking` cannot be resolved. Did you mean one of the following? [`in_spotify_charts`, `in_spotify_playlists`, `in_apple_charts`, `in_apple_playlists`, `in_shazam_charts`].;
'Project [track_name#17, 'in_spotify_ranking]
+- Relation [track_name#17,artist(s)_name#18,artist_count#19,released_year#20,released_month#21,released_day#22,in_spotify_playlists#23,in_spotify_charts#24,streams#25,in_apple_playlists#26,in_apple_charts#27,in_deezer_playlists#28,in_deezer_charts#29,in_shazam_charts#30,bpm#31,key#32,mode#33,danceability_%#34,valence_%#35,energy_%#36,acousticness_%#37,instrumentalness_%#38,liveness_%#39,speechiness_%#40] csv
